# Creación de Pandas DataFrames a partir de peticiones API
En este ejemplo, utilizaremos la API del U.S. Geological Survey para obtener un objeto JSON de datos de terremotos y convertirlo en un `pandas.DataFrame`.

API DEL USGS: https://earthquake.usgs.gov/fdsnws/event/1/

### Obtener datos de la API

In [1]:
import datetime as dt   # para pedir todas las fechas que quiera
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# asegurémonos de que la solicitud fue correcta
response.status_code

200

Una respuesta de 200 significa OK, así que podemos extraer los datos del resultado. Como le pedimos a la API una carga JSON, podemos extraerla de la respuesta con el método `json()`.

### Aislar los datos de la respuesta JSON
Necesitamos comprobar las estructuras de los datos de la respuesta para saber dónde están nuestros datos.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

La API del USGS proporciona información sobre nuestra solicitud en la clave `metadata`. Tenga en cuenta que el resultado será diferente, independientemente del intervalo de fechas que elija, ya que la API incluye una marca de tiempo que indica cuándo se extrajeron los datos:

In [3]:
earthquake_json['metadata']

{'generated': 1702292463000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2023-11-10&endtime=2023-12-10',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.0',
 'count': 9979}

Cada elemento de la matriz JSON `features` es una fila de datos para nuestro marco de datos.

In [4]:
type(earthquake_json['features'])

list

Sus datos serán diferentes dependiendo de la fecha en que ejecute esto.

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 4.7,
  'place': '40 km NE of Barcelona, Philippines',
  'time': 1702166342418,
  'updated': 1702167408040,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us7000lhu1',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us7000lhu1&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 340,
  'net': 'us',
  'code': '7000lhu1',
  'ids': ',us7000lhu1,',
  'sources': ',us,',
  'types': ',origin,phase-data,',
  'nst': 45,
  'dmin': 7.604,
  'rms': 0.65,
  'gap': 126,
  'magType': 'mb',
  'type': 'earthquake',
  'title': 'M 4.7 - 40 km NE of Barcelona, Philippines'},
 'geometry': {'type': 'Point', 'coordinates': [126.7106, 8.3981, 65.36]},
 'id': 'us7000lhu1'}

In [6]:
earthquake_json['features'][0].keys()

dict_keys(['type', 'properties', 'geometry', 'id'])

### Convertir a DataFrame
Necesitamos tomar la sección `properties` de cada entrada del array JSON `features` para crear nuestro dataframe.

In [7]:
pd.DataFrame(earthquake_json['features'])

,type,properties,geometry,id
0,Feature,"{'mag': 4.7, 'place': '40 km NE of Barcelona, ...","{'type': 'Point', 'coordinates': [126.7106, 8....",us7000lhu1
1,Feature,"{'mag': 1.7, 'place': '17 km WNW of Mentone, T...","{'type': 'Point', 'coordinates': [-103.754, 31...",tx2023ycrh
2,Feature,"{'mag': 5, 'place': 'Reykjanes Ridge', 'time':...","{'type': 'Point', 'coordinates': [-34.2664, 57...",us7000lhtz
3,Feature,"{'mag': 3.73, 'place': '146 km NNE of Cruz Bay...","{'type': 'Point', 'coordinates': [-64.4673, 19...",pr2023343003
4,Feature,"{'mag': 0.79, 'place': '6 km WNW of The Geyser...","{'type': 'Point', 'coordinates': [-122.8264999...",nc73973931
...,...,...,...,...
9974,Feature,"{'mag': 2.36, 'place': '5 km N of Pinnacles, C...","{'type': 'Point', 'coordinates': [-121.1335, 3...",nc73960691
9975,Feature,"{'mag': 1.5, 'place': '69 km ESE of Denali Nat...","{'type': 'Point', 'coordinates': [-150.5195, 6...",ak023efasp8t
9976,Feature,"{'mag': 1.8, 'place': '89 km ENE of Ugashik, A...","{'type': 'Point', 'coordinates': [-155.996, 57...",ak023efasbdc
9977,Feature,"{'mag': 0.92, 'place': '12 km NNE of Atka, Ala...","{'type': 'Point', 'coordinates': [-174.1563333...",av91121193


In [8]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
# lo que hago es iterar por cada elemento de la lista "earthquake_json['features']" y cuando el elemento sea "quake['properties']" lo anexo
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,4.70,"40 km NE of Barcelona, Philippines",1702166342418,1702167408040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us7000lhu1,",",us,",",origin,phase-data,",45.0,7.604000,0.65,126.0,mb,earthquake,"M 4.7 - 40 km NE of Barcelona, Philippines"
1,1.70,"17 km WNW of Mentone, Texas",1702165945605,1702167684698,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",tx2023ycrh,",",tx,",",origin,phase-data,",17.0,0.100000,0.50,109.0,mlv,earthquake,"M 1.7 - 17 km WNW of Mentone, Texas"
2,5.00,Reykjanes Ridge,1702165696233,1702167097040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us7000lhtz,",",us,",",origin,phase-data,",54.0,6.986000,0.72,86.0,mb,earthquake,M 5.0 - Reykjanes Ridge
3,3.73,"146 km NNE of Cruz Bay, U.S. Virgin Islands",1702165513570,1702170749040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us7000lhu2,pr2023343003,",",us,pr,",",origin,phase-data,",14.0,1.207700,0.29,306.0,md,earthquake,"M 3.7 - 146 km NNE of Cruz Bay, U.S. Virgin Is..."
4,0.79,"6 km WNW of The Geysers, CA",1702164523680,1702167914212,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc73973931,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",8.0,0.006717,0.02,105.0,md,earthquake,"M 0.8 - 6 km WNW of The Geysers, CA"


### (Opcional) Guardar los datos en un CSV

In [9]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creando_dataframes.ipynb">
        <button style="float: left;">&#8592; Notebook Anterior</button>
    </a>
    <a href="./4-inspeccionando_dataframes.ipynb">
        <button style="float: right;">Siguiente Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>